In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
from functools import reduce
from pathlib import Path

import numpy as np

from pyplume import utils, plotting, constants
from pyplume.postprocess import ParticleResult

In [ ]:
results = {
    "thredds_smoothn": "results/simulation_tijuana_hurrkay_alongshore_2022-10-31T13-18-57-smoothn/particlefile.nc",
    "thredds_nosmoothn": "results/simulation_tijuana_hurrkay_alongshore_2022-11-02T13-36-12-nosmoothn/particlefile.nc",
    "oi": "results/simulation_tijuana_hurrkay_oi_alongshore_2022-10-31T13-18-58/particlefile.nc",
    "henri_hfr": "results/simulation_ny_hurrhenri_hfrnet_2022-11-14T14-35-36/particlefile.nc"
}
to_compare = ["henri_hfr"]
# to_compare = ["thredds_smoothn", "thredds_nosmoothn"]
# coast_lats, coast_lons = utils.load_geo_points("data/coastOR2Mex_tijuana.mat")

domain = constants.TIJUANA_RIVER_DOMAIN
domain = {
    "S": 38.162201,
    "N": 41.520008,
    "W": -75.709632,
    "E": -69.723751
}

res_path = Path(f"results/compare_density_{'_'.join(to_compare)}")
res_path.mkdir(exist_ok=True)

for key in results:
    results[key] = ParticleResult(results[key])

In [ ]:
# common_timestamps = np.intersect1d(results[to_compare[0]].data_vars["time"], results[to_compare[1]].data_vars["time"])
if len(to_compare) <= 1:
    tstamps = results[to_compare[0]].data_vars["time"]
    tstamps = tstamps[~np.isnan(tstamps)]
    common_timestamps = np.sort(np.unique(tstamps))
else:
    common_timestamps = reduce(np.intersect1d, list(map(lambda x: results[key].data_vars["time"], to_compare)))

In [ ]:
for i, t in enumerate(common_timestamps):
    lats1, lons1 = results[to_compare[0]].get_positions_time(t, when="before")
    # lats2, lons2 = results[to_compare[1]].get_positions_time(t, when="before")
    fig, (ax1, ax2) = plotting.carree_subplots((1,2), domain=domain)
    plotting.draw_particle_density(lats1, lons1, bins=100, ax=ax1, pmax=0.7, title=f"{to_compare[0]} cumulative density at\n{t.astype('datetime64[s]')}")
    # plotting.draw_coastline(coast_lats, coast_lons, ax=ax1, c="k")
    # plotting.draw_particle_density(lats2, lons2, bins=100, ax=ax2, pmax=0.7, title=f"{to_compare[1]} cumulative density at\n{t.astype('datetime64[s]')}")
    # plotting.draw_coastline(coast_lats, coast_lons, ax=ax2, c="k")
    strnum = str(i).zfill(3)
    plotting.draw_plt(savefile=res_path / f"plot_{strnum}", fig=fig, fit=True, figsize=(12, 6))

In [ ]:
# save plot as final timestamps

t = common_timestamps[-1]
for i, name in enumerate(to_compare):
    lats, lons = results[name].get_positions_time(t, when="before")
    fig, ax = plotting.carree_subplots((1,1), land=True)
    plotting.draw_particle_density(lats, lons, bins=100, ax=ax, pmax=1, title=f"{name} cumulative density at\n{t.astype('datetime64[s]')}")
    # plotting.draw_coastline(coast_lats, coast_lons, ax=ax, c="k")
    plotting.draw_plt(savefile=res_path / f"density_plot_final_{name}.png", fig=fig, fit=True, figsize=(5, 5))